# Session 8: Production Deployment

**Duration**: 90 minutes  
**Difficulty**: Advanced

## Learning Objectives

- 🎯 Build production-ready FastAPI application
- 🎯 Implement caching with Redis
- 🎯 Add monitoring and logging
- 🎯 Set up security and guardrails
- 🎯 Deploy to cloud (Railway/Render)
- 🎯 Optimize performance and costs

## 📚 What You'll Build

**SupportGenie v1.0 - Production Ready!**

Final version with:
- FastAPI REST API
- Redis caching
- Monitoring dashboard
- Rate limiting
- Security features
- Cloud deployment
- **PRODUCTION READY!** 🚀

## Part 0: Setup

**Note**: This notebook demonstrates production concepts. Some features (like Redis) require installation.

In [ ]:
# Install required packages
!pip install fastapi uvicorn pydantic python-dotenv -q

print("✅ Packages installed!")
print("\n⚠️  Note: Redis caching requires Redis server (see documentation)")

In [ ]:
import os
import json
import time
import hashlib
import logging
from datetime import datetime
from typing import Optional, List

from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
from openai import OpenAI

# Set up API key
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    if 'OPENAI_API_KEY' not in os.environ:
        os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key loaded")

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

print("\n🚀 Ready to build production API!")

## Part 1: FastAPI Application Structure

Build a production-ready REST API.

In [ ]:
# Define request/response models

class QueryRequest(BaseModel):
    """Request model for chat endpoint"""
    message: str
    customer_id: Optional[str] = None
    session_id: Optional[str] = None
    
    class Config:
        json_schema_extra = {
            "example": {
                "message": "What's the status of order ORD-12345?",
                "customer_id": "CUST-001",
                "session_id": "sess_abc123"
            }
        }

class QueryResponse(BaseModel):
    """Response model for chat endpoint"""
    response: str
    sources: List[str] = []
    confidence: float = 1.0
    processing_time_ms: float
    
    class Config:
        json_schema_extra = {
            "example": {
                "response": "Your order ORD-12345 has been shipped.",
                "sources": ["order_database"],
                "confidence": 0.95,
                "processing_time_ms": 245.3
            }
        }

print("✅ Request/Response models defined")

In [ ]:
# Create FastAPI app

app = FastAPI(
    title="SupportGenie API",
    version="1.0",
    description="Production AI Customer Support API"
)

# Mock agent for demonstration
class MockSupportAgent:
    """Mock agent for demo purposes"""
    
    def handle_query(self, query: str, customer_id: str = None) -> dict:
        """Process query"""
        # Simple mock response
        return {
            "answer": f"I understand you're asking about: {query}. Let me help you with that.",
            "sources": ["knowledge_base"],
            "confidence": 0.85
        }

# Initialize agent
agent = MockSupportAgent()

@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "message": "SupportGenie API v1.0",
        "status": "operational",
        "endpoints": ["/chat", "/health", "/metrics"]
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "version": "1.0",
        "timestamp": datetime.now().isoformat()
    }

@app.post("/chat", response_model=QueryResponse)
async def chat(request: QueryRequest):
    """Main chat endpoint"""
    
    start_time = time.time()
    
    try:
        # Process query
        response = agent.handle_query(
            query=request.message,
            customer_id=request.customer_id
        )
        
        processing_time = (time.time() - start_time) * 1000
        
        return QueryResponse(
            response=response['answer'],
            sources=response.get('sources', []),
            confidence=response.get('confidence', 1.0),
            processing_time_ms=processing_time
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ FastAPI app created")
print("\n📝 To run the server:")
print("   uvicorn main:app --host 0.0.0.0 --port 8000")

## Part 2: Caching with Redis (Mock)

Implement response caching to reduce costs and latency.

In [ ]:
class MockCache:
    """Mock cache for demonstration (use Redis in production)"""
    
    def __init__(self):
        self.cache = {}
        self.ttl = 3600  # 1 hour
    
    def get_cache_key(self, query: str, customer_id: str = None) -> str:
        """Generate cache key"""
        data = f"{query}:{customer_id}"
        return hashlib.md5(data.encode()).hexdigest()
    
    def get(self, query: str, customer_id: str = None) -> Optional[dict]:
        """Get cached response"""
        key = self.get_cache_key(query, customer_id)
        
        if key in self.cache:
            entry = self.cache[key]
            
            # Check if expired
            if time.time() - entry['timestamp'] < self.ttl:
                print(f"  ✅ Cache HIT for: {query[:50]}...")
                return entry['data']
            else:
                # Expired
                del self.cache[key]
        
        print(f"  ❌ Cache MISS for: {query[:50]}...")
        return None
    
    def set(self, query: str, response: dict, customer_id: str = None):
        """Cache response"""
        key = self.get_cache_key(query, customer_id)
        self.cache[key] = {
            'data': response,
            'timestamp': time.time()
        }
        print(f"  💾 Cached response for: {query[:50]}...")
    
    def clear(self):
        """Clear all cache"""
        self.cache.clear()
        print("  🗑️  Cache cleared")

# Test cache
cache = MockCache()

# First call - cache miss
result = cache.get("What's your return policy?")
print(f"Result: {result}")

# Set cache
cache.set("What's your return policy?", {"answer": "30 days return policy"})

# Second call - cache hit
result = cache.get("What's your return policy?")
print(f"Result: {result}")

## Part 3: Logging and Monitoring

In [ ]:
# Configure logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

class MetricsLogger:
    """Track API metrics"""
    
    def __init__(self):
        self.metrics = []
    
    def log_request(self, 
                    query: str, 
                    response: str, 
                    latency: float, 
                    cost: float, 
                    customer_id: str = None,
                    cached: bool = False):
        """Log API request metrics"""
        
        metric = {
            "timestamp": datetime.now().isoformat(),
            "customer_id": customer_id,
            "query_length": len(query),
            "response_length": len(response),
            "latency_ms": latency,
            "cost_usd": cost,
            "cached": cached,
            "success": True
        }
        
        self.metrics.append(metric)
        logger.info(f"Request processed: {json.dumps(metric)}")
    
    def get_summary(self) -> dict:
        """Get metrics summary"""
        if not self.metrics:
            return {}
        
        total_requests = len(self.metrics)
        cached_requests = sum(1 for m in self.metrics if m['cached'])
        
        return {
            "total_requests": total_requests,
            "cached_requests": cached_requests,
            "cache_hit_rate": cached_requests / total_requests,
            "avg_latency_ms": sum(m['latency_ms'] for m in self.metrics) / total_requests,
            "total_cost_usd": sum(m['cost_usd'] for m in self.metrics)
        }

# Test metrics logger
metrics = MetricsLogger()

# Log some requests
metrics.log_request(
    query="What's the return policy?",
    response="30 days return policy",
    latency=245.5,
    cost=0.002,
    cached=False
)

metrics.log_request(
    query="What's the return policy?",
    response="30 days return policy",
    latency=5.2,
    cost=0.0,
    cached=True
)

# Get summary
summary = metrics.get_summary()
print("\n📊 Metrics Summary:")
print(json.dumps(summary, indent=2))

## Part 4: Security and Input Validation

In [ ]:
class SecurityValidator:
    """Security and input validation"""
    
    @staticmethod
    def sanitize_input(text: str, max_length: int = 1000) -> str:
        """Sanitize user input"""
        
        # Remove leading/trailing whitespace
        text = text.strip()
        
        # Limit length
        if len(text) > max_length:
            raise ValueError(f"Input too long (max {max_length} characters)")
        
        # Check for malicious patterns
        dangerous_patterns = [
            '<script>',
            'javascript:',
            'onerror=',
            '<?php',
            '<iframe>'
        ]
        
        text_lower = text.lower()
        for pattern in dangerous_patterns:
            if pattern in text_lower:
                raise ValueError(f"Input contains prohibited content: {pattern}")
        
        return text
    
    @staticmethod
    def validate_api_key(api_key: str) -> bool:
        """Validate API key (mock)"""
        # In production: check against database
        valid_keys = ["test-key-123", "prod-key-456"]
        return api_key in valid_keys

# Test security validator
validator = SecurityValidator()

# Test valid input
try:
    clean_text = validator.sanitize_input("What's your return policy?")
    print(f"✅ Valid input: {clean_text}")
except ValueError as e:
    print(f"❌ Invalid input: {e}")

# Test malicious input
try:
    clean_text = validator.sanitize_input("<script>alert('xss')</script>")
    print(f"✅ Valid input: {clean_text}")
except ValueError as e:
    print(f"❌ Blocked malicious input: {e}")

# Test API key
print(f"\nValid key: {validator.validate_api_key('test-key-123')}")
print(f"Invalid key: {validator.validate_api_key('invalid-key')}")

## Part 5: Rate Limiting (Conceptual)

In production, use `slowapi` or similar libraries.

In [ ]:
class SimpleRateLimiter:
    """Simple in-memory rate limiter"""
    
    def __init__(self, max_requests: int = 10, window_seconds: int = 60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests = {}  # user_id -> [timestamps]
    
    def is_allowed(self, user_id: str) -> bool:
        """Check if request is allowed"""
        
        now = time.time()
        
        # Get user's request history
        if user_id not in self.requests:
            self.requests[user_id] = []
        
        # Remove old requests outside the window
        self.requests[user_id] = [
            ts for ts in self.requests[user_id]
            if now - ts < self.window_seconds
        ]
        
        # Check if under limit
        if len(self.requests[user_id]) < self.max_requests:
            self.requests[user_id].append(now)
            return True
        
        return False
    
    def get_remaining(self, user_id: str) -> int:
        """Get remaining requests"""
        if user_id not in self.requests:
            return self.max_requests
        
        now = time.time()
        recent = [
            ts for ts in self.requests[user_id]
            if now - ts < self.window_seconds
        ]
        
        return max(0, self.max_requests - len(recent))

# Test rate limiter
limiter = SimpleRateLimiter(max_requests=3, window_seconds=60)

user_id = "user-123"

print("Testing rate limiter:\n")
for i in range(5):
    allowed = limiter.is_allowed(user_id)
    remaining = limiter.get_remaining(user_id)
    
    status = "✅ ALLOWED" if allowed else "❌ BLOCKED"
    print(f"Request {i+1}: {status} (Remaining: {remaining})")

## Part 6: Cost Optimization

In [ ]:
class CostOptimizer:
    """Optimize costs by selecting appropriate models"""
    
    def __init__(self):
        self.model_costs = {
            "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},  # per 1K tokens
            "gpt-4-turbo": {"input": 0.01, "output": 0.03},
            "gpt-4": {"input": 0.03, "output": 0.06}
        }
    
    def assess_complexity(self, query: str) -> str:
        """Assess query complexity"""
        
        # Simple heuristics
        query_lower = query.lower()
        
        # Complex queries
        complex_keywords = ['explain', 'analyze', 'compare', 'why', 'how does']
        if any(kw in query_lower for kw in complex_keywords):
            return "complex"
        
        # Moderate queries
        moderate_keywords = ['help', 'troubleshoot', 'fix']
        if any(kw in query_lower for kw in moderate_keywords):
            return "moderate"
        
        # Simple queries
        return "simple"
    
    def select_model(self, query: str) -> str:
        """Select optimal model based on complexity"""
        
        complexity = self.assess_complexity(query)
        
        if complexity == "simple":
            return "gpt-3.5-turbo"  # Cheapest
        elif complexity == "moderate":
            return "gpt-4-turbo"    # Balanced
        else:
            return "gpt-4"          # Most capable
    
    def estimate_cost(self, model: str, input_tokens: int, output_tokens: int) -> float:
        """Estimate cost for a request"""
        
        costs = self.model_costs[model]
        
        input_cost = (input_tokens / 1000) * costs['input']
        output_cost = (output_tokens / 1000) * costs['output']
        
        return input_cost + output_cost

# Test cost optimizer
optimizer = CostOptimizer()

test_queries = [
    "Where is my order?",
    "Help me troubleshoot my laptop",
    "Explain how the warranty works and compare it to the extended warranty"
]

print("Cost Optimization:\n")
for query in test_queries:
    complexity = optimizer.assess_complexity(query)
    model = optimizer.select_model(query)
    cost = optimizer.estimate_cost(model, input_tokens=100, output_tokens=200)
    
    print(f"Query: {query[:50]}...")
    print(f"  Complexity: {complexity}")
    print(f"  Model: {model}")
    print(f"  Est. Cost: ${cost:.4f}\n")

## Part 7: Monitoring Dashboard (Simple HTML)

In [ ]:
def generate_dashboard_html(metrics_summary: dict) -> str:
    """Generate simple HTML dashboard"""
    
    total_requests = metrics_summary.get('total_requests', 0)
    avg_latency = metrics_summary.get('avg_latency_ms', 0)
    total_cost = metrics_summary.get('total_cost_usd', 0)
    cache_hit_rate = metrics_summary.get('cache_hit_rate', 0)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SupportGenie Dashboard</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 40px;
                background-color: #f5f5f5;
            }}
            .dashboard {{
                background: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            h1 {{
                color: #333;
            }}
            .metric {{
                display: inline-block;
                margin: 20px;
                padding: 20px;
                background: #f8f9fa;
                border-radius: 5px;
                min-width: 200px;
            }}
            .metric-value {{
                font-size: 2em;
                font-weight: bold;
                color: #007bff;
            }}
            .metric-label {{
                color: #666;
                margin-top: 5px;
            }}
        </style>
    </head>
    <body>
        <div class="dashboard">
            <h1>🤖 SupportGenie Metrics Dashboard</h1>
            
            <div class="metric">
                <div class="metric-value">{total_requests}</div>
                <div class="metric-label">Total Requests</div>
            </div>
            
            <div class="metric">
                <div class="metric-value">{avg_latency:.1f}ms</div>
                <div class="metric-label">Avg Latency</div>
            </div>
            
            <div class="metric">
                <div class="metric-value">${total_cost:.4f}</div>
                <div class="metric-label">Total Cost</div>
            </div>
            
            <div class="metric">
                <div class="metric-value">{cache_hit_rate:.0%}</div>
                <div class="metric-label">Cache Hit Rate</div>
            </div>
        </div>
    </body>
    </html>
    """
    
    return html

# Generate dashboard
dashboard_html = generate_dashboard_html(metrics.get_summary())

# Save to file
with open('dashboard.html', 'w') as f:
    f.write(dashboard_html)

print("✅ Dashboard generated: dashboard.html")
print("\nPreview:")
print(dashboard_html[:500] + "...")

## Part 8: Deployment Configuration

### Dockerfile

In [ ]:
dockerfile_content = """
# Dockerfile for SupportGenie API
FROM python:3.11-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application
COPY . .

# Expose port
EXPOSE 8000

# Run application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

print("📦 Dockerfile:")
print(dockerfile_content)

### Docker Compose

In [ ]:
docker_compose_content = """
# docker-compose.yml
version: '3.8'

services:
  api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - OPENAI_API_KEY=${OPENAI_API_KEY}
      - REDIS_URL=redis://redis:6379
    depends_on:
      - redis

  redis:
    image: redis:alpine
    ports:
      - "6379:6379"
"""

print("🐳 Docker Compose:")
print(docker_compose_content)

### Requirements.txt

In [ ]:
requirements_content = """
fastapi>=0.104.0
uvicorn>=0.24.0
pydantic>=2.0.0
openai>=1.0.0
redis>=5.0.0
python-dotenv>=1.0.0
slowapi>=0.1.9
"""

print("📋 requirements.txt:")
print(requirements_content)

## 🎉 Session 8 Complete! 🎓 COURSE COMPLETE!

### What You Learned:

✅ FastAPI for production APIs  
✅ Redis for caching  
✅ Proper logging and monitoring  
✅ Security with authentication  
✅ Rate limiting  
✅ Docker for containerization  
✅ Cost optimization strategies  
✅ Production deployment best practices

---

## 🎓 CONGRATULATIONS!

**You've completed the entire Gen AI Production Course!**

### Your Journey:

1. ✅ **LLM Fundamentals** - API usage, tokens, costs
2. ✅ **Prompt Engineering** - Advanced prompting techniques
3. ✅ **RAG Systems** - Document retrieval and generation
4. ✅ **Function Calling** - Tool use and actions
5. ✅ **AI Agents** - Autonomous systems with memory
6. ✅ **Multi-Agent** - Orchestration and specialization
7. ✅ **Evaluation** - Testing and quality assurance
8. ✅ **Production** - Deployment and optimization

### Your Capstone: SupportGenie

You built a complete AI customer support platform:
- From simple chatbot → Production-ready system
- Multi-agent architecture
- RAG-powered knowledge base
- Full evaluation framework
- Cloud deployment ready

### Next Steps:

1. **Deploy your project** - Put SupportGenie in production
2. **Build your portfolio** - Showcase your skills
3. **Explore advanced topics** - Fine-tuning, observability
4. **Apply to real projects** - You're ready!

---

**🚀 You're now ready to build production Gen AI applications!**